In [9]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd

# CID : 카테고리(소설 : 1, 에세이 :55889 , 자기계발 : 336)

items = []
lists = []
results = []
KEY = 'ttbdmlcjf10201707001'

CID = 1

for page in range(1,2045): # 페이지 입력
    print(page)
    url = f"https://www.aladin.co.kr/shop/wbrowse.aspx?BrowseTarget=List&ViewRowsCount=50&ViewType=Detail\
    &PublishMonth=0&SortOrder=2&page={page}&Stockstatus=0&PublishDay=84&CustReviewRankStart=0&CustReviewCountStart=0&PriceFilterMax=-1\
    &CID={CID}&SearchOption="
    
    res = requests.get(url) 
    res.raise_for_status() # 정상 200
    soup = BeautifulSoup(res.text, "lxml")
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('ul > li > a.bo3')
    links = []
    for book_link in book_links:
        a= book_link['href']
        links.append(a)
    for link in links:
        res = requests.get(link)
        soup = BeautifulSoup(res.text, "lxml")
        try:
            ISBN = soup.select('div.conts_info_list1 > ul > li')
            ISBN = ISBN[-1].text[-13:]
        except:
            pass
    
        #data = [ISBN]
        results.append(ISBN)

len(results)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106


KeyboardInterrupt: 

In [4]:
import pickle
ISBN336_01_51 = pd.DataFrame(results)
ISBN336_01_51.to_pickle('LINK_LIST_1_01_2045.plk')

In [5]:
# 링크
url_look_list = []

#API 키 발급
KEY = 'ttbdmlcjf10201707001' 

# 링크 저장
for isbn in results:
    url_look = 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey={}&itemIdType=isbn13&ItemId={}&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList'.format(KEY, isbn)
    url_look_list.append(url_look)
print(url_look_list)


['https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbdmlcjf10201707001&itemIdType=isbn13&ItemId=9791198682505&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbdmlcjf10201707001&itemIdType=isbn13&ItemId=9791193262122&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbdmlcjf10201707001&itemIdType=isbn13&ItemId=9791168473690&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbdmlcjf10201707001&itemIdType=isbn13&ItemId=9791166571459&output=JS&Version=20131101&OptResult=authors,reviewList,fulldescription,fulldescription2,Story,categoryIdList', 'https://www.aladin.co.kr/ttb/api/ItemLookUp.aspx?ttbkey=ttbdmlcjf10201707001&itemI

In [6]:
# 저장한 링크에서 책 정보 가져오기
import requests
import pandas as pd

items = []
lists = []

for url_look in url_look_list:
    try:
        response = requests.get(url_look)
        response.raise_for_status()  # 오류가 발생하면 예외를 발생

        text = response.json()  # JSON 응답을 파싱

        item = text.get('item', [])  # 'item' 키가 있는 경우 해당 값을 가져옴
        if item:
            # 도서 정보
            title = item[0].get('title', '')
            author = item[0].get('author', '')
            pubDate = item[0].get('pubDate', '')
            description = item[0].get('description', '')
            isbn13 = item[0].get('isbn13', '')
            categoryID = item[0].get('categoryId', '')
            categoryName = item[0].get('categoryName', '')
            salesPoint = item[0].get('salesPoint', '')
            adult = item[0].get('adult', '')
            rank = item[0].get('customerReviewRank', '')
            fullDes = item[0].get('fullDescription', '')
            fullDes2 = item[0].get('fullDescription2', '')
            story = item[0].get('subInfo', {}).get('story', '')

            # 리뷰 정보
            reviewList = item[0].get('reviewList', [])
            if reviewList:
                reviewrank = reviewList[0].get('reviewRank', 0)
                reviewtitle = reviewList[0].get('title', 0)
                review_w = reviewList[0].get('writer', 0)
            else:
                reviewrank = 0
                reviewtitle = 0
                review_w = 0

            # 리스트에 도서 정보를 추가
            lists.append([title, author, pubDate, description, isbn13, categoryID, categoryName,
                          salesPoint, adult, rank, fullDes, fullDes2, story, reviewrank, reviewtitle, review_w])
    except requests.exceptions.RequestException as e:
        print(f"요청 중 오류 발생: {e}")

# 데이터프레임을 생성
cols = ['title', 'author', 'pubDate', 'description', 'isbn13', 'categoryID', 'categoryName',
        'salesPoint', 'adult', 'rank', 'fullDes', 'fullDes2', 'story', 'reviewrank', 'reviewtitle', 'review_w']
df = pd.DataFrame(lists, columns=cols)

import pickle
df.to_pickle('Self-development_1-50.pkl', protocol=4)
### pkl 이름 ISBN[카테고리 번호]_[페이지 시작]_[페이지 끝] 으로 저장

In [7]:
df

,title,author,pubDate,description,isbn13,categoryID,categoryName,salesPoint,adult,rank,fullDes,fullDes2,story,reviewrank,reviewtitle,review_w
0,나를 소모하지 않는 현명한 태도에 관하여,"마티아스 뇔케 (지은이), 이미옥 (옮긴이)",2024-03-10,"모든 것이 과하게 요구되고, 요란하게 소비되는 시대에 ‘더 현명한 삶의 방식’은 무...",9791198682505,2952,국내도서>자기계발>인간관계>인간관계,78380,False,10,,,,0,0,0
1,일류의 조건,"사이토 다카시 (지은이), 정현 (옮긴이)",2024-03-20,《일류의 조건》이 국내 첫 출간 후 18년 만에 필름출판사에서 복간되었다. 이 책은...,9791193262122,70216,국내도서>자기계발>성공>성공학,82840,False,10,,,,0,0,0
2,세이노의 가르침 (70만 부 기념 빨간 표지) - 피보다 진하게 살아라,세이노(SayNo) (지은이),2023-03-02,"2000년부터 발표된 그의 주옥같은 글들. 독자들이 자발적으로 만든 제본서는 물론,...",9791168473690,70216,국내도서>자기계발>성공>성공학,616810,False,8,,,,0,0,0
3,일을 잘 맡기는 기술 - 일 잘하는 리더보다 일 잘 맡기는 리더가 되라,"모리야 도모타카 (지은이), 정지영 (옮긴이)",2024-03-26,조직과 팀 입장에서 리더 혼자 하는 게 빠른데 왜 일을 맡겨야 하는지 그 이유를 설...,9791166571459,282,국내도서>경제경영>기업 경영>조직/인력/성과관리,15050,False,10,,,,0,0,0
4,겸손의 힘 - 성장과 풍요로운 삶으로 이끄는 현명한 태도,"대릴 반 통게렌 (지은이), 신예용 (옮긴이)",2024-03-20,겸손을 실용적이고 철학적으로 심층 분석한다. 겸손이 어떻게 자신감을 키우고 자신의 ...,9791192389561,70216,국내도서>자기계발>성공>성공학,13930,False,0,,,,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"나쁜 감정에 흔들릴 때 읽는 책 - 불안, 분노, 무력감 뒤에 숨은 진짜 감정을 돌...",권수영 (지은이),2024-02-26,"‘부정적 감정’들은 내면의 진짜 상처를 방어하기 위해 찾아오는 경우가 많으므로, 이...",9791191842647,51514,국내도서>인문학>심리학/정신분석학>교양 심리학,3215,False,10,,,,0,0,0
96,네 안에 잠든 거인을 깨워라 (30주년 기념판),"토니 로빈스 (지은이), 홍석윤 (옮긴이)",2023-09-15,"이 시대 최고의 카운슬러이자 행동변화 전문가인 토니 로빈스. 금연, 다이어트에서부터...",9791166836251,70216,국내도서>자기계발>성공>성공학,29114,False,10,,,,0,0,0
97,하와이 대저택 100일 미라클,하와이 대저택 (지은이),2024-01-25,종합 베스트셀러 1위 《더 마인드》의 저자 ‘하와이 대저택’과 함께하는 100일 쓰...,9791129711519,70216,국내도서>자기계발>성공>성공학,18725,False,7,,,,0,0,0
98,인간 본성의 법칙,"로버트 그린 (지은이), 이지연 (옮긴이)",2019-07-29,세계적인 밀리언셀러 &lt;권력의 법칙&gt;의 저자 로버트 그린이 우리 안에 숨겨...,9791190182560,51514,국내도서>인문학>심리학/정신분석학>교양 심리학,18682,False,9,,,,0,0,0
